In [ ]:
# import anything we need in this notebook
using MsUtils, Compose, Colors, PGFPlots
import LujiaLt
FEM = LujiaLt.FEM;

In [ ]:
# include("../src/LujiaLt.jl"); FEM = LujiaLt.FEM

# Introduction to Approximation

In this notebook we collect the computational examples associated with Chapters 3 and 4 of *Multiscale Simulation*. 

**Remark:** These codes use some elementary functional style constructs. 
These may initially require some getting used to, but in the long run they
make for code that is brief, readable and less likely to contain bugs.
In particular, Julia encourages to avoid `if` statements and instead use
its dispatch mechanism. This is a paradigm that makes for simple and 
elegant code.

## Requirements

* `Compose.jl`; to produce PDFs the latest master needs to be checked out
* `Colors.jl`

## Model

The homogeneous lattice model is
$$
    E(u) = \sum_{\ell \in \Lambda^{\rm hom}} \sum_{i = 1}^3 \phi\big( D_{a_i} u(\ell) \big)
        = \sum_{b \in B^{\rm hom}} \phi( Du_b ).
$$
We will use $\phi(r) = \sin^2( \pi r )$ throughout, which models an 
anti-plane nearest-neighbour pair interaction. 

We can introduce a point defect by introducing additional lattice sites and
associated bonds. A screw dislocation is introduced by adding a *pre-strain* 
to the bonds.  In order to implement only one model, we will describe both
of these within the same framework.

Let $\Lambda$ be a point defect reference configuration and $B$ an
associated set of nearest-neighbour bonds. Specifically we consider two
cases:

* *Homogeneous lattice:* $\Lambda^{\rm hom} = A \mathbb{Z}^2$ and 
    $B^{\rm hom} = \{ (i,j) \in \Lambda^2 : |i-j| = 1 \}$.
* *Interstitial-type defect:* $\Lambda^{\rm int} = \Lambda^{\rm hom} \cup \{\xi\}$, where $\xi = (1/2,0)$ or nearby is the defect site
and $B^{\rm int} = \{(i,j) \in \Lambda^2 : |i-j| \leq 1 \}$.

The pre-strain is a map ${\sf e} : B \to \mathbb{R}$. The energy is now given by
$$
  E(u) = \sum_{b \in B} \phi( {\sf e}_b + Du_b ) - \phi( {\sf e}_b ).
$$

We define the two benchmark problems through specifying $\Lambda, B, {\sf e}$:

* **Point defect case (interstitial)** 
    - $\Lambda = \Lambda^{\rm int}$
    - $B = B^{\rm int}$
    - ${\sf e}_{(i,j)} = 1 - |i-j|$
* **Screw dislocation case** 
    - $\Lambda = \Lambda^{\rm hom}$
    - $B = B^{\rm hom}$
    - ${\sf e}_{(i,j)} = (\frac{1}{2\pi} \arg(i-\xi) - \frac{1}{2\pi}\arg(j-\xi))~{\rm mod}~(-1/2, 1/2]$

More details about these choices can be found in Chapters 3 and 4 of the book.


## ATM Approximation

We first consider the purely atomistic approximation where only the
admissible displacements are restricted. For $R \in \mathbb{N}$ the
computational domain is given by
$$
   \Omega_R = \Lambda \cap [-R+1, R]^2
$$
with Dirichlet conditions (clamped) on the boundary sites. 



In [ ]:
# [1] Preliminaries

# default interstitial position
const ξi = [0.5; 0.0]
const ξs = [0.5; sqrt(3)/4]
# off-centre defect positions, to break symmetry
const ξi_off = [0.3; 0.2]
const ξs_off = [0.4; 0.35]

# a Julia hack that allows us to use dispatch effectively
NoDefectT = Val{:none}; const NoDefect = NoDefectT()
ScrewT = Val{:screw}; const Screw = ScrewT()
InterstitialT = Val{:int}; const Interstitial = InterstitialT()
NoneOrInt = Union{NoDefectT, InterstitialT}
NoneOrScrew = Union{NoDefectT, ScrewT}

"distance of several points from a centre"
dist(X::Matrix, x0) = sqrt(sumabs2(X .- x0, 1))

"default defect cores"
defcore(::InterstitialT) = ξi
defcore(::ScrewT) = ξs
defcore(::NoDefectT) = [0.0;0.0]


# [1] computional domain ------------------------------------------

"""computational domain; returns (x, y) coordinate 
vectors of lattice points"""
domain(R, ::NoneOrScrew; ξ=ξs) = LujiaLt.lattice_ball(R=R, A=LujiaLt.Atri, x0=ξ)[1]
domain(R, ::InterstitialT; ξ=ξi) = [domain(R, NoDefect, ξ=ξ) ξ]


# [3] bonds and operations on bonds -------------------------------

"If v : X → R, then fdiff(v) : Bonds → R storing Dv_b"
fdiff(v::Vector, B::Tuple) = v[B[2]] - v[B[1]]
fdiff(v::Matrix, B::Tuple) = v[:, B[2]] - v[:, B[1]]
fdiff(v, B::Matrix) = fdiff(v, mat2tup(B))

"""bonds within a computational domain: 
find all the Delaunay edges with length <= 1"""
function bonds(X) 
    E = FEM.edges(FEM.Triangulation(X))
    E = E[:, find(sumabs2(fdiff(X, E), 1) .<= 1.1)]
    return mat2tup(E)
end

"bond midpoints"
midpoints(X, B) = 0.5*(X[:, B[1]] + X[:, B[2]])

"distance of bond mid-points from x0"
bonddist(X::Matrix, B::Tuple, ξ) = dist( midpoints(X, B), ξ )
bonddist(X::Matrix, B::Tuple) = bonddist(X, B, [0.0; 0.0])


# [4] prestrain is a function of lattice positions ----------------------------

"compute prestrains"
prestrain(X, B, ::NoneOrInt) = 1.0 - sqrt( sumabs2(fdiff(X, B), 1) )[:]
predictor(X, ::ScrewT; xc = ξs) = predictor_screw(X .- xc)
prestrain(X, B, ::ScrewT; xc = ξs) = prestrain_screw(X .- xc, B)
arg(x, y) = imag(log(x + im * y))
predictor_screw(X) = arg(X[1,:][:], X[2,:][:]) / (2*π)
prestrain_screw(X, B) = mod(fdiff(predictor_screw(X), B) + 0.5, 1) - 0.5

# [5] last aspect of geometry: free indices ----------------------------------

Ifree(X, R; xc = [0.0;0.0]) = find(dist(X, xc) .< R)
Iclamp(X, R; xc = [0.0;0.0]) = find(dist(X, xc) .>= R)
;


### Testing the geometry implementations

We test the geometry implementation by creating an interstitial geometry
and a screw dislocation geometry and plotting both.

In [ ]:
# Interstitial
X = domain(4.0, Interstitial)
B = bonds(X)
ee = prestrain(X, B, Interstitial)
ctx = plot_int(X, B, ee, plotwidth=10cm);

### Implementation of the energies

we have completed all of the geometry assembly and can now turn to the energy.

In [ ]:

# [6] Model type : ------------------------------------------------
#     to collect all the information
#     the construction is produced below
type AtmModel{T}
    R   # domain parameter
    X   # lattice coordinates
    B   # bonds
    e   # pre-strains
    Ifree   # free dofs
    defect::T  # type of defect
end

function AtmModel(; R=5, defect = NoDefect, kwargs...)
    X = domain(R, defect; kwargs...)
    B = bonds(X)
    return AtmModel(R, X, B, prestrain(X, B, defect), Ifree(X, R-1.1), defect)
end

"number of lattice sites in the computational domain"
nsites(m) = size(m.X, 2)
# "number of dofs"
# ndofs(m::AtmModel) = length(m.Ifree)

# [7] Energy ------------------------------------------------------
#     note that could use ForwardDiff.jl to implement
#     the gradient and hessian automatically, but it is very slow
#     it is not difficult to do it by hand:

"NN pair potential (we use the same potential for all experiments)" 
ϕ(r) = sin(r*π).^2
"first derivative of ϕ"
Dϕ(r) = π * sin(2*r*π)
"second derivative of ϕ"
D²ϕ(r) = 2*π^2 * cos(2*r*π)

"potential energy (difference), as function of displacement"
energy(m::AtmModel, U) = sum( ϕ(m.e + fdiff(U[:], m.B)) - ϕ(m.e) )

"gradient of potential energy"
gradient(m::AtmModel, U) = _grad_( m.B, Dϕ(m.e + fdiff(U[:], m.B)), length(U) )
_grad_(B, dϕ, N) = binsum([dϕ; -dϕ], [B[2]; B[1]], N)

"hessian as function of displacement"
hessian(m::AtmModel, U) = _hess_( m.B, D²ϕ(m.e + fdiff(U[:], m.B)), length(U) )
_hess_(B, hϕ, N) = sparse( [B[1]; B[2]; B[1]; B[2]], 
                           [B[1]; B[2]; B[2]; B[1]],  
                           [  hϕ;   hϕ;  -hϕ; -hϕ], N, N )

# [8] Basic Newton solver --------------------------------------------------
#     this is normally not robust enough, but sufficient for our purposes.

function solve(m; x = zeros(nsites(m)), show = false)
    If = m.Ifree
    show && @printf("------Newton Iteration------\n")
    nit = 0
    for nit = 0:12
        ∇E = gradient(m, x)
        show && @printf("%d : %4.2e \n", nit, norm(∇E[If], Inf))
        norm(∇E[If], Inf) < 1e-8 && break
        x[If] = x[m.Ifree] - hessian(m, x)[If, If] \ ∇E[If]
    end
    nit == 12 && warn("the Newton iteration did not converge")
    show && @printf("----------------------------\n")
    return x
end
;

### Testing the implementation

We perform some basic tests to make sure the implementation is correct. 
The basic philosophy of these tests is that, if the energy, gradient and 
hessian are all consistent then the implementation is likely correct. 
Hense we perform a finite difference test. We will test all partial derivatives
for decreasing steps $h$.

Note that
$f'(x) = \frac{f(x+h) - f(x)}{h} + O(h)$, but when we take floating-point errors 
into account, then we obtain
$$
    f'(x) = \frac{f(x+h) - f(x)}{h} + O\left( h + \frac{\epsilon}{h} \right),
$$
where $\epsilon$ denotes floating point accuracy. Hence, as $h$ decreases we first see an improvement but eventually a deterioration of the test agreement.

These tests use `fdtest`, which is implemented in `MsUtils.jl`.

In [ ]:
# run the FD test : change `defect` to :none, :int, :screw
# to test the three implementations
m = AtmModel(R = 10, defect = Interstitial)
println("Test Energy as function of displacement")
fdtest(0.1 * rand(nsites(m)), x->energy(m,x), x->gradient(m,x), x->hessian(m,x))


Our second test concerns the Newton scheme. If it is correctly 
implemented and if gradient and hessian are consistent (we just
tested this), then it should converge in 4-5 iterations at most.

In [ ]:
# change `defect` to {NoDefect, Interstitial, Screw} for the three model problems
m = AtmModel(R = 10, defect=Screw)
solve( m; show=true, x = 0.0 * rand(nsites(m)) );

### Visualise the Solutions

In [ ]:
# interstitial
m = AtmModel(R = 20, defect=Interstitial)
u = solve(m)
Du = fdiff(u, m.B)
axis = [ξi[1]-7; ξi[1]+7; ξi[2]-5; ξi[2]+5]
MsUtils.plot_strain(m.X, m.B, log(abs(m.e+Du)+2e-4); 
                    axis=axis, cmap = colormap("blues"));

In [ ]:
# screw dislocation
m = AtmModel(R = 20, defect=Screw)
u = solve(m)
Du = fdiff(u, m.B)
axis = [ξs[1]-7; ξs[1]+7; ξs[2]-5; ξs[2]+5]
MsUtils.plot_strain(m.X, m.B, log(abs(m.e+Du)+3e-2); 
                    axis=axis, cmap = colormap("blues"));
# u_tot = predictor(m.X, Screw) + u
# MsUtils.plot_displacement(m.X, m.B, u_tot);

### Testing the rate of decay of the solutions

An alternative visualisation, where one can see a lot more is to just plot the decay in terms of distance from the origin.


In [ ]:
# ENVELOPES of strain fields
R = 50    # computational domain radius
Rp = 20   # radius of domain that we actually plot (to avoid boundary effects)
# --- interstitial
mi = AtmModel(R = R, defect=Interstitial, ξ = ξi_off)
ui = solve(mi)
Dui = fdiff(ui, mi.B)
ri = bonddist(mi.X, mi.B, ξi_off)
I = find(ri .<= Rp)
ri = ri[I]; Dui = Dui[I]
# --- screw
ms = AtmModel(R = R, defect=Screw, ξ = ξs_off)
us = solve(ms)
Dus = fdiff(us, ms.B)
rs = bonddist(ms.X, ms.B, ξs_off)
I = find(rs .<= Rp)
rs = rs[I]; Dus = Dus[I]; es = ms.e[I]
# ---- plot
# to make this a nice plot, it takes a little bit of work
xbins = logspace(0, log10(Rp*0.99), 15)
xi, yi = MsUtils.envelope(ri, abs(Dui), xbins)
xs, ys = MsUtils.envelope(rs, abs(Dus), xbins)
xstot, ystot = MsUtils.envelope(rs, abs(es + Dus), xbins)
p = Axis([
    Plots.Linear(xi, yi, style="thick", legendentry="interstitial");
    Plots.Linear(xs, ys, style="thick", legendentry="screw, corrector");
    Plots.Linear(xstot, ystot, style="green, thick", legendentry="screw, total");
    MsUtils.plot_slope(6.0, 20.0, 0.5, -1.0);
    Plots.Node(L"\sim r_b^{-1}", 12.0, 1e-1);
    MsUtils.plot_slope(6.0, 20.0, 0.07, -2.0);
    Plots.Node(L"\sim r_b^{-2}", 10.0, 3e-3);
        ],
        ymode="log", xmode="log",
        xlabel=L"$r_b$",
        ylabel = "strain (envelope)",
        legendPos="south west" )
display(p)


------------

## The LIN Model

To avoid having to implement the Green's function, we implement the 
version of the LIN model described in Exercise 4.3.4, restricting
displacements to a finite computational domain
$$
   \Omega_S = \Lambda \cap B_S
$$
while the energy $E$ is approximated by
$$
   E^{\rm lin}(u) = \sum_{b \in \mathcal{B}^{\rm at}} \phi(e_b+Du_b) - \phi(e_b)
         + \sum_{b \in \mathcal{B} \setminus \mathcal{B}^{\rm at}} 
                 \phi^{\rm lin}(e_b+Du_b) - \phi^{\rm lin}(e_b),
$$
where 
$$
    \phi^{\rm lin}(r) = \phi(0) + \phi'(0) r + \frac12 \phi''(0) r^2.
$$


In [ ]:
# Implementation of the LIN Model

# [1] Lin Model type : ------------------------------------------------
#     
type LinModel{T}
    R   # domain parameter
    S   # domain parameter
    X   # lattice coordinates
    Bat    # nonlinear bonds
    Blin   # quadratic bonds
    eat        # pre-strains
    elin       #  . . .
    Ifree      # free dofs
    defect::T  # type of defect
end


function LinModel(; R=5, S = 0.5 * R^3, defect = NoDefect, ξ=defcore(defect) )
    # atoms
    X = domain(S, defect; ξ=ξ)
    # bonds
    B = bonds(X)
    rB = bonddist(X, B, ξ)
    Iat = find(rB .<= R); Bat = (B[1][Iat], B[2][Iat])
    Ilin = find(rB .> R); Blin = (B[1][Ilin], B[2][Ilin])
    # prestrain
    eat = prestrain(X, Bat, defect)
    elin = prestrain(X, Blin, defect)
    # return model
    return LinModel(R, S, X, Bat, Blin, eat, elin, Ifree(X, S-1.1), defect)
end

# [2] energy

const ϕ1 = Dϕ(0)
const ϕ2 = D²ϕ(0)

ϕlin(r) = ϕ1 * r + 0.5 * ϕ2 * r.^2
Dϕlin(r) = ϕ1 + ϕ2 * r
D²ϕlin(r) = ϕ2 * ones(r)

energy(m::LinModel, U) = (  sum_kbn( ϕ(m.eat + fdiff(U[:], m.Bat)) - ϕ(m.eat) )
                         + sum_kbn( ϕlin(m.elin + fdiff(U[:], m.Blin)) - ϕlin(m.elin) ) )

gradient(m::LinModel, U) =  (  _grad_( m.Bat, Dϕ(m.eat + fdiff(U[:], m.Bat)), length(U) )
                           + _grad_( m.Blin, Dϕlin(m.elin + fdiff(U[:], m.Blin)), length(U) ) )

hessian(m::LinModel, U) = ( _hess_( m.Bat, D²ϕ(m.eat + fdiff(U[:], m.Bat)), length(U) )
                            + _hess_( m.Blin, D²ϕlin(m.elin + fdiff(U[:], m.Blin)), length(U) ) )
;

In [ ]:
# finite-difference check and Newton convergence check
m = LinModel(R = 4, S = 10, defect = Interstitial)
println("Finite-difference test of LIN Model")
fdtest(0.1 * rand(nsites(m)), x->energy(m,x), x->gradient(m,x), x->hessian(m,x))
println("\ntest Newton convergence")
ulin = solve(m, show=true);

### Preliminary error estimate 

We can run a preliminary consistency check, by computing the
error between the ATM and LIN model on the *same* domain
($R^{\rm ATM} = S^{\rm LIN}$). This difference should decay like
$$
    (R^{\rm ATM})^{-1/2} = (S^{\rm LIN})^{-1/2} \approx R^{-3/2},
$$
where $R$ is the inner radius for the LIN model.

The result looks roughly correct, so we can happily move along.

In [ ]:
# we will simulate a screw dislocation with off-centre core
defect = Screw
ξ = ξs_off
# here we store the errors
RR = [3.1; 4.1; 5.1; 7.1; 9.1]
SS = 0.2 * RR.^3 + 1
errinf = Float64[]
erren = Float64[]
NN = Float64[]
# loop over increasing domains
# (this is kind of the limit of what Newton can do!)
for (R, S) in zip(RR, SS)
    atm = AtmModel(R=S, defect=defect, ξ=ξ)
    u_at = solve(atm)
    lin = LinModel(R=R, S=S, defect=defect, ξ=ξ)
    u_lin = solve(lin)
    push!(errinf, norm(u_at - u_lin, Inf) / norm(u_at, Inf))
    push!(erren, norm( fdiff(u_at-u_lin, atm.B) / norm(fdiff(u_at, atm.B)) ))
    push!(NN, length(u_lin))
end



In [ ]:
p = Axis([
    Plots.Linear(NN, errinf, 
        legendentry=L"$\| \bar{u} - \bar{u}^{\rm lin} \|_{\ell^\infty}$");
    Plots.Linear(NN, erren, 
        legendentry=L"$\| D\bar{u} - D\bar{u}^{\rm lin} \|_{\ell^2}$");
    Plots.Linear(NN, 1e-1*NN.^(-1/2), mark="none",
        legendentry=L"$\sim \#\Omega^{-1/2}$");
        ],
        ymode="log", xmode="log",
        xlabel=L"$\# \Omega$",
        ylabel = "relative error",
        legendPos="north east" 
    )


----------------------------------

# A/C Coupling Method

**TODO:** write documentation

In [ ]:
# include("../src/LujiaLt.jl"); FEM = LujiaLt.FEM
# include("./MsUtils.jl")

In [ ]:
# Implementation of the A/C Scheme
# here our carefully constructed abstractions don't
# work so well directly; it is more convenient to
# re-implement some parts from scratch

# [1] Model

type AcModel
    Ra    # atomistic region radius
    Rc    # continuum region radius
    X     # set of nodes (at + c)
    tri   # continuum region triangulation 
    Bat   # atomistic bonds
    Bi    # interface bonds
end


"""core and interface bonds separated"""
function acbonds(Tat)
    # compute the atomistic bonds
    Eat = [ Tat[1,:] Tat[1,:] Tat[2,:]; Tat[2,:] Tat[3,:] Tat[3,:] ]
    Eat = sortcols(sort(Eat, 1))
    occ = ones(Int, size(Eat, 2))
    i = 1
    while i < size(Eat, 2)
        if Eat[:,i] != Eat[:,i+1]
            occ[i] = 1
            i += 1
        else
            occ[i] = 2
            occ[i+1] = 0
            i += 2
        end
    end
    Bat = mat2tup(Eat[:, find(occ .== 2)])
    Bi = mat2tup(Eat[:, find(occ .== 1)])    
    return Bat, Bi
end

function AcModel(; Ra=5, Rc=Ra^2, defect=NoDefect, ξ=defcore(defect))
    # create the atomistic nodes
    Xat = domain(Ra, defect; ξ=ξ)
    # FEM nodes: reuse the implementation in  LujiaLt
    Xc, _ = LujiaLt.radial_nodes(Ra+0.4, Rc; hgrowth=1.0, x0=ξ)
    # concatenate
    X = [Xat Xc]
    # triangulate
    tri = FEM.Triangulation(X)
    
    # we could just get Bat from bonds(Xat), but then we 
    # will have difficulty deciding which bonds are core at
    # and which are interface Instead, we use the triangulation
    # compute all the information needed.
    is_c = ones(FEM.nT(tri))
    for (el, idx) in zip(FEM.elements(tri), 1:FEM.nT(tri))
        x = tri.X[:, el.t]
        # if all indices of the element are in Xat
        #    and if all edges are <= 1, then this is is an atomistic element
        if (maximum(el.t) <= size(Xat,2)) &&
            (maximum(sumabs2(x[:,[2;3;1]] - x, 1)) <= 1.01)
            is_c[idx] = 0
        end
    end
    # compute the atomistic bonds from this
    Bat, Bi = acbonds(tri.T[:, find(is_c .== 0)])
    
    # now get rid of the atomistic elements all-together
    tri.T = tri.T[:, find(is_c .== 1)]
    
    return AcModel(Ra, Rc, X, tri, Bat, Bi)
end
;

In [ ]:
# [2] A/C Energy



In [ ]:
# We can check-out what the A/C geometry looks like.
#  this gives us also some confidence that the implementation is correct
ac = AcModel(Ra=3.2, Rc=7, defect=Interstitial)
MsUtils.plot_acgeom(ac, plotwidth=10cm);